In [26]:
from core import read_mmseqs_results
import pandas as pd
import os

In [4]:
seq_id_names = pd.read_parquet('../data/interim/seq_id_names.pq')
go_df = pd.read_parquet('../data/interim/seq_id_go_processes.pq')


In [ ]:
conda_commands = ['conda run -n beaker mmseqs']

In [14]:
query_fa = '../data/interim/secreted_proteins.faa'

In [ ]:
if 'secreted_proteins.faa' not in os.listdir('../data/interim/'):
    os.system('cat secreted_proteins/*.fasta > ' + query_fa)

In [ ]:
working_dir = '../data/interim/mmseqs_working_dir'

In [ ]:
query_db = working_dir + 'secreted_proteins_db'

In [ ]:
os.sytem(' '.join(conda_commands +
                  ['create_db', 
                   query_fa, 
                   query_db, 
                   '--dbtype 1']))

In [ ]:
target_db = working_dir + 'unique_seqs_db'
out_db = working_dir + 'secreted_proteins_refseq_search'

In [ ]:
os.system(' '.join(conda_commands +
                   ['search', query_db, target_db, 
                    out_db, '../data/tmp',
                    '--threads 48',
                    '-c 0.8', '--min-seq-id 0.3', 
                    '-a']))

In [ ]:
target_out = '../data/interim/secreted_proteins_refseq_search.txt'

In [ ]:
os.system(' '.join(conda_commands +
                   ['convertalis', query_db, 
                    target_db, out_db, target_out, 
                    '--threads 48', '--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov
                    ']))

In [2]:
secreted_protein_result = read_mmseqs_results('../data/interim/secreted_proteins_refseq_search.txt')

In [35]:
gold_standard_homologs = secreted_protein_result[(secreted_protein_result['fident'] > 0.9) &
                                                 (secreted_protein_result['qcov'] > 0.9) & 
                                                 (secreted_protein_result['tcov'] > 0.9)]

In [41]:
unique_homologs = gold_standard_homologs['target'].drop_duplicates()

In [42]:
len(unique_homologs)

2692

In [37]:
secreted_protein_rows = seq_id_names['seq_id'].isin(unique_homologs)

In [38]:
seq_id_names.loc[secreted_protein_rows, 'name'].value_counts().head(50)

name
Hcp family type VI secretion system effector                        265
hypothetical protein                                                254
serralysin family metalloprotease                                   138
type VI secretion system effector Hcp                                79
type VI secretion system tube protein Hcp                            75
IS481-like element IS481 family transposase                          64
type VI secretion system tip protein VgrG                            57
calcium-binding protein                                              39
type I methionyl aminopeptidase                                      34
type III effector                                                    29
type VI secretion system effector Hcp1                               29
M48 family metallopeptidase                                          26
MaoC family dehydratase                                              25
four-carbon acid sugar kinase family protein               

In [39]:
go_rows = go_df['seq_id'].isin(unique_homologs)

In [40]:
go_df.loc[go_df['seq_id'].isin(unique_homologs), 'go_process'].value_counts()

go_process
proteolysis                                                                  143
protein secretion by the type III secretion system                            77
protein secretion by the type VI secretion system                             42
carbohydrate metabolic process                                                30
protein modification process                                                  26
cellular protein modification process                                         11
secretion by the type IV secretion system                                     11
negative regulation of protein secretion                                       7
organic phosphonate catabolic process                                          5
nodulation                                                                     2
organic substance metabolic process                                            2
methionine biosynthetic process                                                2
ribosome biogenes

In [43]:
unique_homologs.to_csv('../data/interim/secreted_homologs.csv', index=False)